# 6103 Tean project

## Drouvas Panagiotis, Dimos Giorgos, Kontoudakis Nikos, Vergas Panagiotis

In [7]:
#library import
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

### Part 1.

In [8]:
# specifically for part 1
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.metrics import silhouette_samples
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA

#future selection
from sklearn.feature_selection import mutual_info_classif

In [9]:
#insert the data

# the names might be icorect
collumnNames =  [
    "duration", "protocol_type", "service", "flag", "src_bytes", "dst_bytes", "land", 
    "wrong_fragment", "urgent", "hot", "num_failed_logins", "logged_in", 
    "num_compromised", "root_shell", "su_attempted", "num_root", "num_file_creations", 
    "num_shells", "num_access_files", "num_outbound_cmds", "is_host_login", 
    "is_guest_login", "count", "srv_count", "serror_rate", "srv_serror_rate", 
    "rerror_rate", "srv_rerror_rate", "same_srv_rate", "diff_srv_rate", 
    "srv_diff_host_rate", "dst_host_count", "dst_host_srv_count", "dst_host_same_srv_rate", 
    "dst_host_diff_srv_rate", "dst_host_same_src_port_rate", "dst_host_srv_diff_host_rate", 
    "dst_host_serror_rate", "dst_host_srv_serror_rate", "dst_host_rerror_rate", 
    "dst_host_srv_rerror_rate", "label"
]

allData =pd.read_csv('kddcup.data_10_percent', header=None, names = collumnNames)
y_labels = allData.iloc[:, -1]

In [12]:
# one hot encoding
data = pd.get_dummies(allData.iloc[:, 0:41], columns=["protocol_type", "service", "flag"])

# normalize
scaler = MinMaxScaler()
data = scaler.fit_transform(data)

# reduce dimensions
pca = PCA(n_components=14)
data = pca.fit_transform(data)

X_train, X_test, y_train, y_test = train_test_split(data, y_labels, test_size=0.85, random_state=13)

In [5]:
information=mutual_info_classif(X_train,y_train)
print('Information=',information)

Information= [1.03548534 1.02495383 0.9785964  1.00465346 0.92187281 0.92294568
 0.9069116  0.94811116 0.91220932 0.89139552 0.82464304 0.83340694
 0.85389313 0.84791029]


## DBSCAN

In [13]:
evaluationMetricsDBSCAN = []
data = X_train.copy()
print(len(data))
# for i in np.arange(2, 20, 1):
#     # for x in [0.03, 0.05, 0.07, 0.1, 0.2, 0.5, 1, 2]:  
#     for x in [1, 2, 5, 7, 9]:  

# db = DBSCAN(eps=x, min_samples=i).fit(X_train)

db = DBSCAN().fit(data)

74103


In [ ]:


# core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
# core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_
# n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
# # if(len(set(labels))> 1 and n_clusters_ > 1):
silhouette_values = silhouette_samples(data, y_train)
# # evaluationMetricsDBSCAN.append({'min_samples': i, 'eps': x, 'silhouette': round(np.mean(silhouette_values), 3), 'expected number of clusters': n_clusters_})
print('silhouette: ', round(np.mean(silhouette_values), 3))
            
# metricsDF = pd.DataFrame(evaluationMetricsDBSCAN)
# sortedMetrics= metricsDF.sort_values(by=['silhouette'], ascending= False)
# sortedMetrics[:4]